# Setup

The package is imported as `cr` in this chunk. Sub-modules can be called using:
- `cr.pp` (functions in the `crispr/processing` directory scripts)
- `cr.ax` (functions in the `crispr/processing` directory scripts)
- `cr.pl` (functions in the `crispr/visualization` directory scripts)
- `cr.tl` (functions in the `crispr/utils` directory scripts)

You can access functions this way directly (e.g., `cr.tl.print_counts()`) if they are "exposed," i.e., listed in the `__init__.py` script under a given sub-module (e.g., `crispr/processing/__init__.py`).

Other functions or objects you may need to import (e.g., the default perturbation layer name, using `from crispr.analysis.perturbations import layer_perturbation`); however, most or all functions are exposed.

There are also certain `Crispr` class initialization arguments defined here. No need to change anything***; these arguments are all just to identify columns in the CRISPR screening data and such.

You will specify arguments to your liking in the next chunk of code under the "Options" section.

---

\*** One exception may be `col_cell_type` to "leiden" or "majority_voting" if you want one of those as the default cell type label used in downstream processes, though you can always just specify something different using the `col_cell_type` argument in a given method.

In [6]:
%load_ext autoreload
%autoreload 2

import crispr as cr
from crispr.class_sc import Omics
from anndata import AnnData
import scanpy as sc
import copy
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from examples import config

col_sample_id = "orig.ident"
kws_harmony = dict(plot_convergence=True, random_state=1618)
kws_init = dict(assay=None, assay_protein=None, col_sample_id=None,
                col_gene_symbols="gene_symbols",
                col_cell_type="predicted_labels",
                col_perturbed="perturbation",
                col_guide_rna="feature_call",
                col_num_umis="num_umis",
                col_condition="target_gene_name",
                key_control="NT", key_treatment="KD")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
cr.tl.print_pretty_dictionary(kws_cluster)

method_cluster="leiden"
kws_umap=dict(min_dist = "0.3")
kws_neighbors=None
resolution="1.2"
kws_pca=dict(n_comps = "20", use_highly_variable = "True")


# File Path

Here I specify the file paths for HH03-HH06. Each dataset is specified via `{"directory": <FILE_PATH>}` to signify that it's a CellRanger output.

I specify the `file_path` argument via code below in order to detect where your data are relative to the working directory of this script; however, in practice, you can specify paths directly/manually. 

For example, you can specify as a dictionary of multiple files in order to :integrate the samples into one object using Harmony.

```
file_path = {"HH03_normoxia_noMDP": {"directory": "/home/elizabeth/elizabeth/crispr/examples/data/crispr-screening/HH03"},
             "HH04_normoxia_MDP": {"directory": "/home/elizabeth/elizabeth/crispr/examples/data/crispr-screening/HH04"},
             "HH05_hypoxia_noMDP": {"directory": "/home/elizabeth/elizabeth/crispr/examples/data/crispr-screening/HH05"},
             "HH06_hypoxia_MDP": {"directory": "/home/elizabeth/elizabeth/crispr/examples/data/crispr-screening/HH06"}}
self = cr.Crispr(file_path)
```

If you use this method, you'd need to specify certain initialization arguments differently. Refer to `crispr/examples/crispr_integration.ipynb` for an example.

---

**In this notebook**, for ease of examining minimally-processed objects separately, we will create multiple objects, using each element of the `file_path_list` as the `file_path` argument for each dataset. This will return a list of individual objects (instantiations of the class defined in `crispr/class_crispr.py`) for each dataset.

```
file_path_list = [{"directory": "/home/elizabeth/elizabeth/crispr/examples/data/crispr-screening/HH03"},
                  {"directory": "/home/elizabeth/elizabeth/crispr/examples/data/crispr-screening/HH04"},
                  {"directory": "/home/elizabeth/elizabeth/crispr/examples/data/crispr-screening/HH05"},
                  {"directory": "/home/elizabeth/elizabeth/crispr/examples/data/crispr-screening/HH06"}]
selves = [cr.Crispr(file_path)]
```

In [2]:
file_path = ["HH03_normoxia_noMDP", "HH04_normoxia_MDP",
             "HH05_hypoxia_noMDP", "HH06_hypoxia_MDP"]
file_path = dict([(x, {"directory": os.path.join(
    config.DIR, "crispr-screening", x.split("_")[0])}) for x in file_path])
file_path_list = [file_path[x] for x in file_path]

# Options

These first options you may want to fiddle with.

## Guide RNA Processing

In [8]:
kws_process_guide_rna = dict(feature_split = "|", guide_split = "-", 
                             key_control_patterns = ["CTRL"], 
                             remove_multi_transfected = True)

## scRNA-seq Pre-Processing & Clustering

You don't necessarily need to change these, but you can if you want to fiddle with the options. 

More options are available for greater customization. See the docstring in the functions in `crispr/processing/preprocessing.py` (the `cr.pp` module, if you imported the package using `import crispr as cr` as is done in the first chunk in this notebook) and `crispr/analysis/clustering.py`. 

The main wrapper function for preprocessing is `cr.pp.process_data()`. This function calls additional functions in the same script (and some others in the visualization module, `crispr/visualization/preprocessing.py`)

In [14]:
kws_umap = dict(min_dist=0.3)
kws_pp = {"kws_hvg": {"min_mean": 0.0125, "max_mean": 10, 
                      "min_disp": 0.5, "flavor": "cell_ranger"}, 
          "target_sum": 10000.0, 
          "cell_filter_pmt": [0, 15], 
          "cell_filter_ngene": [200, 7000], 
          "cell_filter_ncounts": [500, 60000], 
          "gene_filter_ncell": [3, None], 
          "kws_scale": "z", "regress_out": None, "kws_umap": kws_umap}
kws_cluster = {"method_cluster": "leiden", "kws_umap": kws_umap, 
               "kws_neighbors": None, "resolution": 1.2,
               "kws_pca": {"n_comps": 20, "use_highly_variable": True}}

# Data

In [ ]:
selves = [cr.Crispr(f, **kws_init, kws_process_guide_rna=kws_grna) 
          for f in file_path_list] # create object